**importing packages**

In [4]:
import fiftyone as fo
from glob import glob
import os
from tqdm.notebook import tqdm

**Dawnloading data**

coco dataset

In [3]:
from pycocotools.coco import COCO
import requests
import os
import random
from tqdm import tqdm

# Path to the COCO annotations file
ann_file = r'C:\Users\Tarek\fiftyone\coco-2017\raw\instances_train2017.json'

# Initialize COCO API
coco = COCO(ann_file)

# Specify the category you want (e.g., "car")
cat_ids = coco.getCatIds(catNms=['car'])
img_ids = coco.getImgIds(catIds=cat_ids)

# Shuffle and select 5000 image IDs
random.shuffle(img_ids)
img_ids = img_ids[:600]

# Load the relevant images
images = coco.loadImgs(img_ids)

# Create a directory to save the images
output_dir = 'car_images_600'
os.makedirs(output_dir, exist_ok=True)

loading annotations into memory...
Done (t=27.29s)
creating index...
index created!


In [3]:
import json

# Specify the path to your JSON file
json_file_path = 'cocoo.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)  # Save the JSON content to a Python variable

In [12]:
downloaded_images = os.listdir('car_images_5000')

In [13]:
images = [image for image in data if image['file_name'] not in downloaded_images]

In [ ]:
# Download images with progress bar
for img in tqdm(images, desc="Downloading images", unit="image"):
    img_data = requests.get(img['coco_url']).content
    with open(os.path.join(output_dir, img['file_name']), 'wb') as handler:
        handler.write(img_data)

print(f"Downloaded {len(images)} car images.")

In [17]:
data == images

False

**!!!!!!check first the cocoo.json is the same as images when i run the cell**

In [6]:
import json
from pycocotools.coco import COCO
import os
from tqdm import tqdm

# Path to the COCO annotations file
ann_file = r'C:\Users\Tarek\fiftyone\coco-2017\raw\instances_train2017.json'

# Initialize COCO API
#coco = COCO(ann_file)

# Directory where images are saved
output_dir = 'car_images_600'
downloaded_files = os.listdir(output_dir)
downloaded_file_names = [os.path.splitext(f)[0] for f in downloaded_files]

# Map filenames to COCO image IDs and gather required data with progress bar
img_ids = []
images = []

for img_id in tqdm(coco.getImgIds(), desc="Mapping filenames to image IDs", unit="image"):
    img_info = coco.loadImgs(img_id)[0]
    if os.path.splitext(img_info['file_name'])[0] in downloaded_file_names:
        img_ids.append(img_id)
        images.append({
            "id": img_info['id'],
            "file_name": img_info['file_name'],
            "width": img_info['width'],
            "height": img_info['height']
        })

# Gather annotations for these images with progress bar
annotations = []
cat_ids = coco.getCatIds(catNms=['car'])
ann_ids = coco.getAnnIds(imgIds=img_ids, catIds=cat_ids, iscrowd=False)
anns = coco.loadAnns(ann_ids)

for ann in tqdm(anns, desc="Gathering annotations", unit="annotation"):
    annotations.append({
        "id": ann['id'],
        "image_id": ann['image_id'],
        "category_id": ann['category_id'],
        "segmentation": ann['segmentation'],
        "bbox": ann['bbox'],
        "area": ann['area'],
        "iscrowd": ann['iscrowd']
    })

# Define categories
categories = [
    {
        "id": cat_ids[0],  # This should be the ID of the "car" class
        "name": "car",
        "supercategory": "vehicle"
    }
]

# Combine all into the final JSON structure
coco_format = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

# Save the JSON file
output_json = os.path.join(output_dir, 'car_annotations_coco600.json')
with open(output_json, 'w') as f:
    json.dump(coco_format, f)

print("COCO-format JSON file created successfully.")


Mapping filenames to image IDs:   0%|          | 0/118287 [00:00<?, ?image/s]

Gathering annotations: 100%|██████████| 2116/2116 [00:00<00:00, 423149.96annotation/s]


COCO-format JSON file created successfully.


In [ ]:
# Load annotations for the selected images
ann_ids = coco.getAnnIds(imgIds=img_ids, catIds=cat_ids, iscrowd=False)
anns = coco.loadAnns(ann_ids)

# Save annotations (optional)
# You can save these annotations in a file or process them as needed
ann_output_path = os.path.join(output_dir, 'car_annotations.json')
with open(ann_output_path, 'w') as f:
    import json
    json.dump(anns, f)

print("Annotations saved.")


open images dataset

In [ ]:
dataset = fo.zoo.load_zoo_dataset(
    "open-images",
    splits="train",
    label_types="segmentations",
    # max_samples=50,
)

print('data downloaded')

Loading dataset

In [46]:
# The directory containing the dataset to import
dataset_dir = r"C:\Users\Tarek\fiftyone\open-images-v7\train"

# The type of the dataset being imported
dataset_type = fo.types.dataset_types.OpenImagesV7Dataset  # for example

# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    label_types=["segmentations"]
)

 100% |█████████████| 15003/15003 [26.7m elapsed, 0s remaining, 7.3 samples/s]      


**getting labels for downloaded images**

In [4]:
def extract_id_from_image_path(path):
    id = path.split('\\')[-1][0:16]
    return id
def extract_id_from_mask(path):
    mask_id  = path.split('\\')[-1]
    return mask_id

In [3]:
ROOT_DIR = r'C:\Users\Tarek\fiftyone\open-images-v7\train'
images_path = glob(os.path.join(ROOT_DIR,'data')+'/*')
#images_path = images_path[40:60]
image_ids = [extract_id_from_image_path(path) for path in images_path]

In [4]:
label_folders = glob(os.path.join(ROOT_DIR,'labels','masks')+'/*')
labels_paths = []
i = 1
for folder in tqdm(label_folders, desc='printing folders'):
    print(f'working on the  {i}th folder and folder path is :{folder}')
    labels = glob(folder+'/*')
    labels_filtred = [path for path in labels if extract_id_from_image_path(path) in image_ids and 'm0k4j' in path ]
    labels_paths.extend(labels_filtred)
    i+=1
print('labels extracted successfully')

printing folders:   0%|          | 0/16 [00:00<?, ?it/s]

working on the  1th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\0
working on the  2th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\1
working on the  3th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\2
working on the  4th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\3
working on the  5th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\4
working on the  6th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\5
working on the  7th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\6
working on the  8th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\7
working on the  9th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\8
working on the  10th folder and folde

In [11]:
labels_id = [extract_id_from_image_path(path) for path in labels_paths]

In [ ]:
from collections import Counter


# Count the occurrences of each element
element_counts = Counter(labels_id)

# Print the counts
print(element_counts)


In [12]:
import os
import shutil
def copy_files(file_paths, destination_directory):
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
        
    for file_path in tqdm(file_paths, desc='copying files'):
        if os.path.isfile(file_path):
            try:
                shutil.copy(file_path, destination_directory)
                #print(f"Copied {file_path} to {destination_directory}")
            except Exception as e:
                print(f"Failed to copy {file_path}: {e}")
        else:
            print(f"{file_path} does not exist or is not a file.")

**converting from binary masks to coco format**

In [25]:
import datetime
import os
import numpy as np
import json
from PIL import Image
from tqdm.notebook import tqdm
from pycococreator.pycococreatortools import pycococreatortools

INFO = {
    "description": "Car_Tracking_2024",
    "url": "None",
    "version": "1.0",
    "year": 2024,
    "contributor": "Tarek",
    "date_created": datetime.datetime.now(datetime.UTC).isoformat(' ')
}

LICENSES = [{"id": 1, "name": "Zewail"}]
CATEGORIES = [{'id': 1, 'name': 'm0k4j', 'supercategory': 'none'}]

In [26]:
coco_output = {
    "info": INFO,
    "licenses": LICENSES,
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}


In [21]:
image_paths = images_path
annotation_files = labels_paths

In [22]:
print(f'the number of images is: {len(image_paths)}')
print(f'the number of masks is: {len(annotation_files)}')

the number of images is: 15003
the number of masks is: 32532


In [23]:
image_id = 1
for image_path in tqdm(image_paths, desc='images preprocessing'):
    image = Image.open(image_path)
    image_filename = image_path.split('\\')[-1]
    image_info = pycococreatortools.create_image_info(image_id, os.path.basename(image_filename), image.size)
    coco_output["images"].append(image_info)
    image_id += 1

images preprocessing:   0%|          | 0/15003 [00:00<?, ?it/s]

In [ ]:
from PIL import Image

segmentation_id = 1
for annotation_filepath in tqdm(annotation_files, desc='labels preprocessing'):
    image_id = [f for f in coco_output['images'] if f['file_name'][0:16] == extract_id_from_image_path(annotation_filepath)][0]['id']
    annotation_filename = annotation_filepath.split('\\')[-1]
    class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename][0]
    category_info = {'id': class_id, 'is_crowd': 'crowd' in annotation_filename}

    # Load and check mask size
    mask_image = Image.open(annotation_filepath).convert('1')
    
    # Resize mask if needed
    if mask_image.size != (coco_output['images'][image_id-1]['width'], coco_output['images'][image_id-1]['height']):
        #print(f"Resizing mask from {mask_image.size} to {(coco_output['images'][image_id-1]['width'], coco_output['images'][image_id-1]['height'])}")
        mask_image = mask_image.resize((coco_output['images'][image_id-1]['width'], coco_output['images'][image_id-1]['height']), Image.NEAREST)
    
    binary_mask = np.asarray(mask_image).astype(np.uint8)

    # Debugging: Print sizes to ensure they match
    #print(f"Processing image ID: {image_id}, Image size: {coco_output['images'][image_id-1]['width']}x{coco_output['images'][image_id-1]['height']}, Mask size: {mask_image.size}")

    annotation_info = pycococreatortools.create_annotation_info(segmentation_id, image_id, category_info, binary_mask, mask_image.size, tolerance=2)

    if annotation_info is not None:
        coco_output["annotations"].append(annotation_info)
        segmentation_id += 1


In [76]:
coco_output['categories'] = [{'id': 1, 'name': 'car', 'supercategory': 'none'}]

import json
with open('data.json', 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

In [2]:
import json

# Specify the path to your JSON file
json_file_path = 'test/data.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)  # Save the JSON content to a Python variable

**From COCO to YOLO**

In [28]:
import json
import os
import shutil
import yaml
def convert_to_yolo(input_images_path, input_json_path, output_labels_path):
    # Open JSON file containing image annotations
    f = open(input_json_path)
    data = json.load(f)
    f.close()

    # Create directories for output images and labels
    #os.makedirs(output_images_path, exist_ok=True)
    os.makedirs(output_labels_path, exist_ok=True)

    # List to store filenames
    file_names = []
    for filename in os.listdir(input_images_path):
        if filename.endswith(".jpg"):
            #source = os.path.join(input_images_path, filename)
            #destination = os.path.join(output_images_path, filename)
            #shutil.copy(source, destination)
            file_names.append(filename)

    # Function to get image annotations
    def get_img_ann(image_id):
        return [ann for ann in data['annotations'] if ann['image_id'] == image_id]

    # Function to get image data
    def get_img(filename):
        return next((img for img in data['images'] if img['file_name'] == filename), None)

    # Iterate through filenames and process each image
    for filename in tqdm(file_names, desc='process each image'):
        img = get_img(filename)
        img_id = img['id']
        img_w = img['width']
        img_h = img['height']
        img_ann = get_img_ann(img_id)

        # Write normalized polygon data to a text file
        if img_ann:
            with open(os.path.join(output_labels_path, f"{os.path.splitext(filename)[0]}.txt"), "a") as file_object:
                for ann in img_ann:
                    current_category = ann['category_id'] - 1
                    polygon = ann['segmentation'][0]
                    normalized_polygon = [format(coord / img_w if i % 2 == 0 else coord / img_h, '.6f') for i, coord in enumerate(polygon)]
                    file_object.write(f"{current_category} " + " ".join(normalized_polygon) + "\n")

In [29]:
input_image_paths = r'C:\Users\Tarek\fiftyone\open-images-v7\train\data'
input_json_path = r'test\data.json'
output_labels_path = r'Labels'
convert_to_yolo(input_image_paths, input_json_path, output_labels_path)

process each image:   0%|          | 0/15003 [00:00<?, ?it/s]

**Splittin Dataset**

In [20]:
all_images_path = glob(r'C:\Users\Tarek\fiftyone\open-images-v7\train\data' +'/*')
all_labels_path = glob(r'D:\self-grinding\deeplearning\Project\object-tracking\Labels' +'/*')
print(len(all_images_path))
print(len(all_labels_path))

15003
15003


In [21]:
all_ids = [extract_id_from_image_path(path) for path in all_images_path]

In [22]:
from sklearn.model_selection import train_test_split
train_ids, val_ids = train_test_split(all_ids, test_size=0.2, shuffle=True)

In [23]:
images_root_dir = r'C:\Users\Tarek\fiftyone\open-images-v7\train\data'
labels_root_dir = r'Labels'

In [24]:
train_image_paths = [os.path.join(images_root_dir,f'{id}.jpg') for id in train_ids]
val_image_paths = [os.path.join(images_root_dir,f'{id}.jpg') for id in val_ids]
train_label_paths = [os.path.join(labels_root_dir,f'{id}.txt') for id in train_ids]
val_label_paths = [os.path.join(labels_root_dir,f'{id}.txt') for id in val_ids]

In [25]:
copy_files(train_image_paths, r'data\images\train')
copy_files(val_image_paths, r'data\images\val' )
copy_files(train_label_paths, r'data\labels\train')
copy_files(val_label_paths, r'data\labels\val' )

copying files:   0%|          | 0/12002 [00:00<?, ?it/s]

copying files:   0%|          | 0/3001 [00:00<?, ?it/s]

copying files:   0%|          | 0/12002 [00:00<?, ?it/s]

copying files:   0%|          | 0/3001 [00:00<?, ?it/s]